  # Robotik Assignment 04

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# 4-1

Play the provided bagfile in a loop:

# 4-2

Extract the intrinsic parameters 

In [ ]:
#This command saves and then renames the parameters appropriately

DF = pd.read_csv("/sensor_data.csv") #Replace this address
DF.drop(columns=['%time', 'field.header.seq', 'field.header.stamp',
                 'field.header.frame_id'],inplace=True)
DF.rename(columns={"field.K0":"fx","field.K2":"cx","field.K4":"fy",
                   "field.K5":"cy","field.D0":"k1",
                   "field.D1":"k2","field.D2":"t1","field.D3":"t2",
                   "field.D4":"k3"}, inplace= True)
parameter = DF[["fx", "fy", "cx","cy", "k1", "k2", "t1", "t2", "k3"]]

Extracting the intrinsic parameters and distortion coefficients. 

In [ ]:
parameter.iloc[0]  #extracting the required parameters.

#  4-3

Use the threshold function from OpenCV to turn your gray image into binary image. 
Draw black rectangles over the remaining white pixels in the binary image 
and Publish your image. 

In [ ]:
# import opencv   #Arman's Code

from imutils import contours
from skimage import measure
import numpy as np
import argparse
import imutils
import cv2
import matplotlib.pyplot as plt

# Read image
src = cv2.imread("images/frame0007.jpg", cv2.IMREAD_GRAYSCALE)

# Set threshold and maxValue
thresh = 230   #230 is nice threshold!
maxValue = 255
 
# Basic threshold example
th, dst = cv2.threshold(src, thresh, maxValue, cv2.THRESH_BINARY);


#Painting over the extra white spaces

painted = cv2.rectangle(dst,(0,0),(800,100),(0,0,0), -1)
painted = cv2.rectangle(dst,(250,250),(490,400),(0,0,0), -1) 
painted = cv2.rectangle(dst,(0,350),(800,800),(0,0,0), -1) 

plt.imshow(painted, cmap='gray')

cv2.imwrite("images/threshold.jpg", painted); 




from skimage import measure


labels = measure.label(painted, neighbors = 8, background = 0)
mask = np.zeros(painted.shape, dtype="uint8")

for label in np.unique(labels):
    if label == 0:
        continue
        
        
# otherwise, construct the label mask and count the
# number of pixels 
labelMask = np.zeros(painted.shape, dtype="uint8")
labelMask[labels == label] = 255
numPixels = cv2.countNonZero(labelMask)
 
# if the number of pixels in the component is sufficiently
# large, then add it to our mask of "large blobs"
if numPixels > 300:
    mask = cv2.add(mask, labelMask)

#Part2
# find the contours in the mask, then sort them from left to
# right
cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
#cnts = contours.sort_contours(cnts)[0]
 
# loop over the contours
for (i, c) in enumerate(cnts):
# draw the bright spot on the image
    (x, y, w, h) = cv2.boundingRect(c)
    ((cX, cY), radius) = cv2.minEnclosingCircle(c)
    cv2.circle(image, (int(cX), int(cY)), int(radius),
    (0, 0, 255), 3)
    cv2.putText(image, "#{}".format(i + 1), (x, y - 15),
    cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
 
    #show the output image
cv2.imshow("Image", painted)
cv2.waitKey(0)


#  4-4

Now, divide the thresholded image into 6 regions so that every region has one marker. For every region extract all the white pixels and calculate the average position to get the center of the marking.

# 4-5